In [ ]:
import sys
import os
import plotly.graph_objs as go
import plotly.express as px
import numpy as np
import scipy.signal
import scipy.ndimage
sys.path.append('..')
import gridijkstra
os.getpid()

# Example 1: Random hilly terrain

In [ ]:
np.random.seed(123)
nf = 151
w = np.random.normal(size=(213 + 2 * nf, 177 + 2 * nf))
ws = scipy.ndimage.gaussian_filter(w, 20)
# wg = np.linalg.norm(np.array(np.gradient(ws)), axis=0)

costs = 1.0 + (3 * (ws - ws.min()) / (ws.max() - ws.min())) ** 2

In [ ]:
px.imshow(costs.T, origin='lower')

In [ ]:
stencil_4 = [
    (1, 0),
    (0, 1),
    (-1, 0),
    (0, -1),
]

stencil_8 = stencil_4 + [
    (1, 1),
    (-1, 1),
    (1, -1),
    (-1, -1),
]

stencil_24 = stencil_8 + [
    (2, -2), (2, -1), (2, 0), (2, 1), (2, 2),
    (1, -2),                          (1, 2),
    (0, -2),                          (0, 2),
    (-1, -2),                         (-1, 2),
    (-2, -2), (-2, -1), (-2, 0), (-2, 1), (-2, 2),
]

In [ ]:
total_4, path_4, length_4 = gridijkstra.plan(
    costs,
    (10, 10),
    (307, 417),
    stencil_4,
    return_path=True,
    return_length=True,
)

total_8, path_8, length_8 = gridijkstra.plan(
    costs,
    (10, 10),
    (307, 417),
    stencil_8,
    return_path=True,
    return_length=True,
)

total_24, path_24, length_24 = gridijkstra.plan(
    costs,
    (10, 10),
    (307, 417),
    stencil_24,
    return_path=True,
    return_length=True,
)

In [ ]:
print( 'Neighbors     length      total')
print(f'4             {length_4:6.1f}     {total_4:5.5}')
print(f'8             {length_8:6.1f}     {total_8:5.5}')
print(f'24            {length_24:6.1f}     {total_24:5.5}')

In [ ]:
fig = px.imshow(costs.T, origin='lower', color_continuous_scale='gray_r', zmax=costs.max() / 3)
fig.add_scatter(x=path_4[:, 0], y=path_4[:, 1], name='4 neighbors')
fig.add_scatter(x=path_8[:, 0], y=path_8[:, 1], name='8 neighbors')
fig.add_scatter(x=path_24[:, 0], y=path_24[:, 1], name='24 neighbors')
fig.layout.coloraxis.showscale = False
fig

# Example 2: Calculate upland

In [ ]:
upland_stencil = stencil_8.copy()

In [ ]:
upland_gradients = {
    s: (np.roll(costs, s, axis=(0, 1)) - costs) / np.linalg.norm(s)
    for s in upland_stencil
}
upland_costs = {
    s: np.where(ug <= 0.0, 1.0, np.inf)
    for s, ug in upland_gradients.items()
}
# Correct for incorrect (but convenient) use of np.roll
for ug in upland_costs.values():
    ug[0, :] = np.inf
    ug[:, 0] = np.inf
    ug[-1, :] = np.inf
    ug[:, -1] = np.inf

In [ ]:
ii, jj = np.meshgrid(range(costs.shape[0]), range(costs.shape[1]), indexing='ij')
ij1 = np.array((ii.flatten(), jj.flatten())).T
ij0 = np.zeros_like(ij1)
ij0[:, 0] = 225
ij0[:, 1] = 316

total_costs = gridijkstra.plan(upland_costs, ij0, ij1)
total_costs_2d = total_costs.reshape(costs.shape[0], costs.shape[1])

In [ ]:
px.imshow(total_costs_2d.T, origin='lower')

In [ ]:
costs_dup = costs.copy()
costs_dup[np.isinf(total_costs_2d)] = np.nan
f = px.imshow(costs.T, origin='lower', color_continuous_scale='gray')
f.add_scatter(x=[ij0[0, 0]], y=[ij0[0, 1]])
f.add_heatmap(z=costs_dup.T, colorscale='jet')